In [1]:
import requests
import random
import pandas as pd 
from tqdm import tqdm
from bs4  import BeautifulSoup
from cred import YOUR_API_KEY
import json 
from tqdm import tqdm
import time

import ast

/home/brocoleta/.local/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
YOUR_API_KEY = YOUR_API_KEY

In [3]:
df = pd.read_csv('precios_de_combustibles.csv')

In [4]:
df['Municipio'].unique()

array(['BOGOTA  D.C.'], dtype=object)

# Clean Data 

In [5]:
df[df['Periodo']==2022]['Mes'].max()

2

In [6]:
df3 =  df[(df['Periodo']==2022)]

In [7]:
len(df3)

1368

In [8]:
df3['Municipio'].unique()

array(['BOGOTA  D.C.'], dtype=object)

In [9]:
df3['Dir2'] = df3['Direccion'].apply(lambda x : x.replace('#','No'))
df3['Full_Address'] = df3.apply(lambda x : x['Direccion'] + ', ' + x['Municipio'].capitalize() + ', Colombia' , axis = 1)

In [10]:
df3['Full_Address'].nunique()

376

In [11]:
df3['Full_Address'][0]

'AVENIDA CARRERA 7 No. 123A-48, Bogota  d.c., Colombia'

In [12]:
estaciones =  df3[['Nombre_comercial','Direccion','Full_Address']].drop_duplicates(subset = 'Nombre_comercial')
estaciones.reset_index(inplace = True)
estaciones.drop(columns = 'index',inplace = True)

In [13]:
estaciones['Full_Address'][0]

'AVENIDA CARRERA 7 No. 123A-48, Bogota  d.c., Colombia'

In [14]:
address = estaciones['Full_Address'][0]
url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={YOUR_API_KEY}'
response = requests.get(url).json()

In [15]:
def Get_Coords(address,YOUR_API_KEY):
    
    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={YOUR_API_KEY}'
    
    try : 
        
        response = requests.get(url).json()
        CleanAddress = response['items'][0]['title'].upper()
        LAT = response['items'][0]['position']['lat']
        LNG = response['items'][0]['position']['lng']
        
        results = [CleanAddress,LAT,LNG]
        
    except :
        
        results = ['Not Found','NA','NA']
        
    
    return results 

In [16]:
Get_Coords(address,YOUR_API_KEY)

['AVENIDA CARRERA 7 123A-48, 110111 BOGOTÁ, D.C., COLOMBIA',
 4.69921,
 -74.02878]

# Extracción de coordenadas 

In [17]:
coordenadas = []
tiempo = [1,2,2.5]

for i in tqdm(range(len(estaciones['Full_Address'])),colour = 'green'):
    
    try :
        
        R = Get_Coords(estaciones['Full_Address'][i],YOUR_API_KEY)
        geo_source = R[1],R[2]
        coordenadas.append(geo_source)
        
    except :
        
        coordenadas.append('Error')
        
    
    time.sleep(random.choices(tiempo)[0])
    
        

  0%|          | 0/376 [00:00<?, ?it/s]

100%|██████████| 376/376 [17:02<00:00,  2.72s/it]


In [25]:
estaciones['Coords'] = coordenadas

In [26]:
len(df3)

1368

In [27]:
estaciones.head(2)

,Nombre_comercial,Direccion,Full_Address,Coords
0,SUPER ESTACION DE SERVICIO TEXACO 10,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"
1,ESTACION DE SERVICIO ROXI,Carrera 24 No 7-11,"Carrera 24 No 7-11, Bogota d.c., Colombia","(4.60425, -74.09239)"


In [28]:
df_final = pd.merge(df3,estaciones[['Full_Address','Coords']],how = 'inner' , on = 'Full_Address')

In [29]:
df_final.head(3)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Dir2,Full_Address,Coords
0,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,GASOLINA CORRIENTE OXIGENADA,9860,A,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"
1,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,BIODIESEL EXTRA,9780,A,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"
2,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,GASOLINA EXTRA OXIGENADA,17250,A,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"


In [30]:
df_final = pd.merge(df3,estaciones[['Full_Address','Coords']],how = 'inner' , on = 'Full_Address')

In [31]:
df_final.to_csv('DATASET_FINAL.csv',index = False)